### Faceted Product Search
This is a demo of product search scenario based on facets (topics) mined from Amazon reviews.<br/>

Following demo is based on a subset of data from the following dataset <br/>
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz

This demo used metapy library to build, host & query from inverted index [https://github.com/meta-toolkit/metapy](https://github.com/meta-toolkit/metapy)

In [25]:
import pandas as pd

import math
import metapy
import pytoml
import re

### Build Index
- We indexed mined topics, goal is to do faceted search. instead of title search
- Uses inbuilt BM25 ranker using default hyper parameters

In [32]:
config_file = 'config.toml'
index = metapy.index.make_inverted_index(config_file)
ranker = metapy.index.OkapiBM25()

### Load Product metadata

In [33]:
product_meta = pd.read_json('../data/product_sentiments.product_aggregated.slim.json')

### Query Index
Issue few queries againt the index

In [34]:
sample_query = 'best noise cancelling'
index_query = metapy.index.Document()
index_query.content(sample_query)

In [37]:
top_docs = ranker.score(index, index_query, num_results=50)
result_productids = []

for num, (d_id, _) in enumerate(top_docs):
    content = index.metadata(d_id).get('content')
    if content is not None:
        product_id = re.split(r'\t+', content)[0]
        result_productids.append(product_id)

### Search Results for the given query

In [38]:
pd.set_option('display.max_colwidth', -1)
search_results = pd.DataFrame(columns=['product_id','product_title'])

for product in result_productids:
    product_id = str(product)
    found_df = product_meta[product_meta['product_id'] == product_id][['product_id', 'product_title']]
    if found_df.shape[0] > 0:
        search_results.append(found_df[['product_id', 'product_title']])
        search_results = pd.concat([search_results, found_df])
    #print(found_df[['product_id', 'product_title']])

search_results

,product_id,product_title
304,B00EWJHRMY,Bose QuietComfort 15 Acoustic Noise Cancelling Headphones - Limited Edition (Discontinued by Manufacturer)
232,B0094S36RI,Logitech 982-000079 UE 6000 Headphones
40,B000GFDC7C,Bose? QuietComfort? 3 Acoustic Noise Cancelling? Headphones
397,B00M1NEUA0,"Bose QuietComfort 25 Headphones (wired, 3.5mm)"
268,B00BSXRBGE,NoiseHush Active Noise-Cancelling Over-Ear Headphones - Black / Silver
444,B00QPHW63G,Symphonized NRG Premium Genuine Wood In-ear Noise-isolating Headphones|Earbuds|Earphones with Microphone
56,B000VWOL3O,Zune Premium Headphones for Zune 4GB
366,B00JHKSL1O,Panasonic eneloop pro NEW High Capacity Power Pack
359,B00IYA31AY,Powerbeats 2 Wireless In-Ear Headphone - White
399,B00M5A7FGU,Bose SoundTrue In-Ear Headphones
